In [ ]:
import time
import json
import random
import base64
from pathlib import Path
import os

import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import requests
import numpy as np

In [ ]:
class ImageEncoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.encoder = torch.nn.Sequential(
            *list(self.resnet.children())[:-2],
            torch.nn.AdaptiveAvgPool2d(1),
            torch.nn.Flatten()
        )

    def forward(self, x):
        x = self.encoder(x)
        return x

In [ ]:
def preprocess(image):
  device = torch.device('cuda')
  mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
  std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

  transform = transforms.Compose([
        #transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
  ])

  image_transform = transform(image)

  return torch.Tensor(image_transform).unsqueeze(0).to(device)

In [ ]:
def matching_hash_two_pictures(tensor_one, tensor_two):
    count_match = 0

    for i in range(len(tensor_one)):
        if tensor_one[i] == tensor_two[i]:
            count_match += 1

    print(f"Совпадение соответствующих feature map двух картинок = {100 * (count_match/len(tensor_one))} %")

In [ ]:
def compare_two_pictures(path_image_one, path_image_two, model, change_one_pixel=False):
    image = Image.open(path_image_one).convert('RGB')
    preprocessed = preprocess(image)
    output_image_one = model(preprocessed)
    output_image_one = output_image_one.detach().cpu().numpy()

    print(output_image_one.shape)

    image = Image.open(path_image_two).convert('RGB')
    preprocessed = preprocess(image)

    if change_one_pixel:
        preprocessed[0][0][0][0] = 0

    output_image_two = model(preprocessed)
    output_image_two = output_image_two.detach().cpu().numpy()
    print(output_image_two.shape)

    cos = torch.nn.CosineSimilarity(dim=1)
    # print(torch.Tensor(output_image_one))
    # print(torch.Tensor(output_image_one).shape)
    cos = cos(torch.Tensor(output_image_one), torch.Tensor(output_image_two))

    # print(output_image_one[0].shape)
    # print(output_image_two[0].shape)

    print(f"Косинусное расстояние между двумя картинками равно : {cos[0]}")
    matching_hash_two_pictures(output_image_one[0], output_image_two[0])

In [ ]:
def preprocess_other(image):
  device = torch.device('cuda')
  mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
  std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

  transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
  ])

  image_transform = transform(image)

  return torch.Tensor(image_transform).unsqueeze(0).to(device)

In [ ]:
def compare_two_pictures_other(path_image_one, path_image_two, model, change_one_pixel=False):
    image = Image.open(path_image_one).convert('RGB')
    preprocessed = preprocess_other(image)
    output_image_one = model(preprocessed)
    output_image_one = output_image_one.detach().cpu().numpy()

    #image = torch.Tensor(np.array(Image.open(path_image_two).convert('RGB'))).unsqueeze(0).to(device)
    image = Image.open(path_image_two).convert('RGB')
    preprocessed = preprocess(image)
    # mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
    # std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

    # transform = transforms.Compose([
    #     transforms.ToTensor(),
    #     #transforms.Normalize(mean, std)
    # ])

    # image = transform(image)
    # image = torch.Tensor(image).unsqueeze(0).to(device)

    if change_one_pixel:
        preprocessed[0][0][0][0] = 0

    output_image_two = model(preprocessed)
    output_image_two = output_image_two.detach().cpu().numpy()

    cos = torch.nn.CosineSimilarity(dim=1)
    # print(torch.Tensor(output_image_one))
    # print(torch.Tensor(output_image_one).shape)
    cos = cos(torch.Tensor(output_image_one), torch.Tensor(output_image_two))

    # print(output_image_one[0].shape)
    # print(output_image_two[0].shape)

    print(f"Косинусное расстояние между двумя картинками равно : {cos[0]}")
    matching_hash_two_pictures(output_image_one[0], output_image_two[0])

In [ ]:
def compare_two_pictures_other1(path_image_one, path_image_two, model, change_one_pixel=False):
    image = Image.open(path_image_one).convert('RGB')
    preprocessed = preprocess_other(image)
    output_image_one = model(preprocessed)
    output_image_one = output_image_one.detach().cpu().numpy()

    #image = torch.Tensor(np.array(Image.open(path_image_two).convert('RGB'))).unsqueeze(0).to(device)
    image = Image.open(path_image_two).convert('RGB')
    preprocessed = preprocess_other(image)
    # mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
    # std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

    # transform = transforms.Compose([
    #     transforms.ToTensor(),
    #     #transforms.Normalize(mean, std)
    # ])

    # image = transform(image)
    # image = torch.Tensor(image).unsqueeze(0).to(device)

    if change_one_pixel:
        preprocessed[0][0][0][0] = 0

    output_image_two = model(preprocessed)
    output_image_two = output_image_two.detach().cpu().numpy()

    cos = torch.nn.CosineSimilarity(dim=1)
    # print(torch.Tensor(output_image_one))
    # print(torch.Tensor(output_image_one).shape)
    cos = cos(torch.Tensor(output_image_one), torch.Tensor(output_image_two))

    # print(output_image_one[0].shape)
    # print(output_image_two[0].shape)

    print(f"Косинусное расстояние между двумя картинками равно : {cos[0]}")
    matching_hash_two_pictures(output_image_one[0], output_image_two[0])

In [ ]:
model = ImageEncoder()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.eval()
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 197MB/s]


In [ ]:
# тестирование на 100 картинках


In [ ]:
!git clone https://github.com/Alextezinn/hash512.git

Cloning into 'hash512'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 105 (delta 0), reused 105 (delta 0), pack-reused 0
Receiving objects: 100% (105/105), 1.44 MiB | 10.79 MiB/s, done.


In [ ]:
def new_matching_hash_two_pictures(tensor_one, tensor_two):
    count_match = 0

    for i in range(len(tensor_one)):
        if tensor_one[i] == tensor_two[i]:
            count_match += 1
    print(count_match)
    print(f"Совпадение соответствующих feature map двух картинок = {100 * (count_match/len(tensor_one))} %")
    return 100 * (count_match/len(tensor_one))

In [ ]:
def compare_two_pictures_px(path_image_one, path_image_two, model, count_px):
    image = Image.open(path_image_one).convert('RGB')
    preprocessed = preprocess(image)
    output_image_one = model(preprocessed)
    output_image_one = output_image_one.detach().cpu().numpy()

    print(output_image_one.shape)

    image = Image.open(path_image_two).convert('RGB')
    preprocessed = preprocess(image)

    # shape [1, 3, 224, 224]

    for i in range(count_px):
        preprocessed[0][0][i*10][i*10] = 0
        preprocessed[0][1][i*10][i*10] = 0
        preprocessed[0][2][i*10][i*10] = 0

    output_image_two = model(preprocessed)
    output_image_two = output_image_two.detach().cpu().numpy()
    print(output_image_two.shape)

    cos = torch.nn.CosineSimilarity(dim=1)
    # print(torch.Tensor(output_image_one))
    # print(torch.Tensor(output_image_one).shape)
    cos = cos(torch.Tensor(output_image_one), torch.Tensor(output_image_two))

    # print(output_image_one[0].shape)
    # print(output_image_two[0].shape)

    print(f"Косинусное расстояние между двумя картинками равно : {cos[0]}")
    return cos[0], new_matching_hash_two_pictures(output_image_one[0], output_image_two[0])

In [ ]:
hash = 0
cos_ = 0
path = Path("/content/hash512/images_100")
for filename in os.listdir(path):
    a, b = compare_two_pictures_px(path / filename, path / filename, model, 1)
    cos_ += float(a)
    hash += b

print(hash / 100)
print(cos_ / 100)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9999976754188538
81
Совпадение соответствующих feature map двух картинок = 15.8203125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9999977350234985
74
Совпадение соответствующих feature map двух картинок = 14.453125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9998930096626282
33
Совпадение соответствующих feature map двух картинок = 6.4453125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9997707009315491
23
Совпадение соответствующих feature map двух картинок = 4.4921875 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9999990463256836
71
Совпадение соответствующих feature map двух картинок = 13.8671875 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9999014735221863
19
Совпадение соответствующих feature map двух картинок = 3.7109375 %
(1, 512)
(1, 512)
Косинусное расстояни

In [ ]:
hash = 0
cos_ = 0
path = Path("/content/hash512/images_100")
for filename in os.listdir(path):
    a, b = compare_two_pictures_px(path / filename, path / filename, model, 3)
    cos_ += float(a)
    hash += b

print(hash / 100)
print(cos_ / 100)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9991762042045593
76
Совпадение соответствующих feature map двух картинок = 14.84375 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9999954700469971
74
Совпадение соответствующих feature map двух картинок = 14.453125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.999197244644165
31
Совпадение соответствующих feature map двух картинок = 6.0546875 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.997766375541687
18
Совпадение соответствующих feature map двух картинок = 3.515625 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.999928891658783
71
Совпадение соответствующих feature map двух картинок = 13.8671875 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9993699193000793
20
Совпадение соответствующих feature map двух картинок = 3.90625 %
(1, 512)
(1, 512)
Косинусное расстояние между 

In [ ]:
hash = 0
cos_ = 0
path = Path("/content/hash512/images_100")
for filename in os.listdir(path):
    a, b = compare_two_pictures_px(path / filename, path / filename, model, 5)
    cos_ += float(a)
    hash += b

print(hash / 100)
print(cos_ / 100)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9990684390068054
75
Совпадение соответствующих feature map двух картинок = 14.6484375 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9999768733978271
74
Совпадение соответствующих feature map двух картинок = 14.453125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9991226196289062
31
Совпадение соответствующих feature map двух картинок = 6.0546875 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9978119134902954
18
Совпадение соответствующих feature map двух картинок = 3.515625 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.999420702457428
68
Совпадение соответствующих feature map двух картинок = 13.28125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9987521767616272
19
Совпадение соответствующих feature map двух картинок = 3.7109375 %
(1, 512)
(1, 512)
Косинусное расстояние ме

In [ ]:
hash = 0
cos_ = 0
path = Path("/content/hash512/images_100")
for filename in os.listdir(path):
    a, b = compare_two_pictures_px(path / filename, path / filename, model, 10)
    cos_ += float(a)
    hash += b

print(hash / 100)
print(cos_ / 100)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9990720152854919
75
Совпадение соответствующих feature map двух картинок = 14.6484375 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9997474551200867
73
Совпадение соответствующих feature map двух картинок = 14.2578125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.999115526676178
30
Совпадение соответствующих feature map двух картинок = 5.859375 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.997560441493988
17
Совпадение соответствующих feature map двух картинок = 3.3203125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9989897012710571
68
Совпадение соответствующих feature map двух картинок = 13.28125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9985366463661194
19
Совпадение соответствующих feature map двух картинок = 3.7109375 %
(1, 512)
(1, 512)
Косинусное расстояние ме

In [ ]:
hash = 0
cos_ = 0
path = Path("/content/hash512/images_100")
for filename in os.listdir(path):
    a, b = compare_two_pictures_px(path / filename, path / filename, model, 15)
    cos_ += float(a)
    hash += b

print(hash / 100)
print(cos_ / 100)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9989192485809326
74
Совпадение соответствующих feature map двух картинок = 14.453125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9997178316116333
73
Совпадение соответствующих feature map двух картинок = 14.2578125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.999129056930542
30
Совпадение соответствующих feature map двух картинок = 5.859375 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9972987771034241
17
Совпадение соответствующих feature map двух картинок = 3.3203125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9966977834701538
59
Совпадение соответствующих feature map двух картинок = 11.5234375 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9985429048538208
18
Совпадение соответствующих feature map двух картинок = 3.515625 %
(1, 512)
(1, 512)
Косинусное расстояние м

In [ ]:
mkdir images_rotation_90

In [ ]:
def compare_two_pictures_cos(path_image_one, path_image_two, model):
    image = Image.open(path_image_one).convert('RGB')
    preprocessed = preprocess(image)
    output_image_one = model(preprocessed)
    output_image_one = output_image_one.detach().cpu().numpy()

    print(output_image_one.shape)

    image = Image.open(path_image_two).convert('RGB')
    preprocessed = preprocess(image)

    output_image_two = model(preprocessed)
    output_image_two = output_image_two.detach().cpu().numpy()
    print(output_image_two.shape)

    cos = torch.nn.CosineSimilarity(dim=1)
    # print(torch.Tensor(output_image_one))
    # print(torch.Tensor(output_image_one).shape)
    cos = cos(torch.Tensor(output_image_one), torch.Tensor(output_image_two))

    # print(output_image_one[0].shape)
    # print(output_image_two[0].shape)

    print(f"Косинусное расстояние между двумя картинками равно : {cos[0]}")
    matching_hash_two_pictures(output_image_one[0], output_image_two[0])
    return float(cos[0])

In [ ]:
path = Path("/content/hash512/images_100")
for filename in os.listdir(path):
  im = Image.open(path / filename).convert('RGB')
  im_rotate = im.rotate(90)
  im_rotate.save(Path("/content/images_rotation_90") / filename, quality=95)

In [ ]:
cos_ = 0
path1 = Path("/content/hash512/images_100")
path2 = Path("/content/images_rotation_90")
for filename in os.listdir(path1):
    cos_ += compare_two_pictures_cos(path1 / filename, path2 / filename, model)

print(cos_ / 100)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.7105175852775574
Совпадение соответствующих feature map двух картинок = 3.3203125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.7434599995613098
Совпадение соответствующих feature map двух картинок = 4.296875 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.6263516545295715
Совпадение соответствующих feature map двух картинок = 0.5859375 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.7747554779052734
Совпадение соответствующих feature map двух картинок = 0.390625 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.7086989283561707
Совпадение соответствующих feature map двух картинок = 4.296875 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.6011229157447815
Совпадение соответствующих feature map двух картинок = 0.390625 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками

In [ ]:
mkdir images_rotation_180

In [ ]:
path = Path("/content/hash512/images_100")
for filename in os.listdir(path):
  im = Image.open(path / filename).convert('RGB')
  im_rotate = im.rotate(180)
  im_rotate.save(Path("/content/images_rotation_180") / filename, quality=95)

In [ ]:
cos_ = 0
path1 = Path("/content/hash512/images_100")
path2 = Path("/content/images_rotation_180")
for filename in os.listdir(path1):
    cos_ += compare_two_pictures_cos(path1 / filename, path2 / filename, model)

print(cos_ / 100)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.8722093105316162
Совпадение соответствующих feature map двух картинок = 7.421875 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.6701005697250366
Совпадение соответствующих feature map двух картинок = 1.953125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.7486675977706909
Совпадение соответствующих feature map двух картинок = 1.3671875 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.8204772472381592
Совпадение соответствующих feature map двух картинок = 1.171875 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.8709620833396912
Совпадение соответствующих feature map двух картинок = 7.6171875 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.67848140001297
Совпадение соответствующих feature map двух картинок = 0.78125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками ра

In [ ]:
mkdir images_rotation_270

In [ ]:
path = Path("/content/hash512/images_100")
for filename in os.listdir(path):
  im = Image.open(path / filename).convert('RGB')
  im_rotate = im.rotate(270)
  im_rotate.save(Path("/content/images_rotation_270") / filename, quality=95)

In [ ]:
cos_ = 0
path1 = Path("/content/hash512/images_100")
path2 = Path("/content/images_rotation_270")
for filename in os.listdir(path1):
    cos_ += compare_two_pictures_cos(path1 / filename, path2 / filename, model)

print(cos_ / 100)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.762994110584259
Совпадение соответствующих feature map двух картинок = 5.6640625 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.7199424505233765
Совпадение соответствующих feature map двух картинок = 4.1015625 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.6520212292671204
Совпадение соответствующих feature map двух картинок = 0.5859375 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.648316502571106
Совпадение соответствующих feature map двух картинок = 0.9765625 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.7419946789741516
Совпадение соответствующих feature map двух картинок = 3.3203125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.599269449710846
Совпадение соответствующих feature map двух картинок = 0.1953125 %
(1, 512)
(1, 512)
Косинусное расстояние между двумя картинкам

In [ ]:
def new_preprocess_other(image, width, height):
  device = torch.device('cuda')
  mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
  std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

  transform = transforms.Compose([
        transforms.Resize((width, height)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
  ])

  image_transform = transform(image)

  return torch.Tensor(image_transform).unsqueeze(0).to(device)

In [ ]:
cos_ = 0

path = Path("/content/hash512/images_100")
for filename in os.listdir(path):
  image = Image.open(path / filename).convert('RGB')
  # уменьшаем в 2 раза
  preprocessed = new_preprocess_other(image, 75, 75)
  output_image_one = model(preprocessed)
  output_image_one = output_image_one.detach().cpu().numpy()

  image = Image.open(path / filename).convert('RGB')
  preprocessed = preprocess(image)

  output_image_two = model(preprocessed)
  output_image_two = output_image_two.detach().cpu().numpy()

  cos = torch.nn.CosineSimilarity(dim=1)
  cos = cos(torch.Tensor(output_image_one), torch.Tensor(output_image_two))
  print(f"Косинусное расстояние между двумя картинками равно : {cos[0]}")
  cos_ += float(cos[0])

print(cos_ / 100)

Косинусное расстояние между двумя картинками равно : 0.6904900074005127
Косинусное расстояние между двумя картинками равно : 0.6649866104125977
Косинусное расстояние между двумя картинками равно : 0.6687912940979004
Косинусное расстояние между двумя картинками равно : 0.6338293552398682
Косинусное расстояние между двумя картинками равно : 0.5178884267807007
Косинусное расстояние между двумя картинками равно : 0.637270450592041
Косинусное расстояние между двумя картинками равно : 0.7112370729446411
Косинусное расстояние между двумя картинками равно : 0.6761659979820251
Косинусное расстояние между двумя картинками равно : 0.719896137714386
Косинусное расстояние между двумя картинками равно : 0.6506193280220032
Косинусное расстояние между двумя картинками равно : 0.737788200378418
Косинусное расстояние между двумя картинками равно : 0.6083250641822815
Косинусное расстояние между двумя картинками равно : 0.63172847032547
Косинусное расстояние между двумя картинками равно : 0.73196280002593

In [ ]:
cos_ = 0

path = Path("/content/hash512/images_100")
for filename in os.listdir(path):
  image = Image.open(path / filename).convert('RGB')
  # увеличиваем в 3 раза
  preprocessed = new_preprocess_other(image, 450, 450)
  output_image_one = model(preprocessed)
  output_image_one = output_image_one.detach().cpu().numpy()

  image = Image.open(path / filename).convert('RGB')
  preprocessed = preprocess(image)

  output_image_two = model(preprocessed)
  output_image_two = output_image_two.detach().cpu().numpy()

  cos = torch.nn.CosineSimilarity(dim=1)
  cos = cos(torch.Tensor(output_image_one), torch.Tensor(output_image_two))
  print(f"Косинусное расстояние между двумя картинками равно : {cos[0]}")
  cos_ += float(cos[0])

print(cos_ / 100)

Косинусное расстояние между двумя картинками равно : 0.6744785904884338
Косинусное расстояние между двумя картинками равно : 0.7703930735588074
Косинусное расстояние между двумя картинками равно : 0.7595457434654236
Косинусное расстояние между двумя картинками равно : 0.7971145510673523
Косинусное расстояние между двумя картинками равно : 0.7955452799797058
Косинусное расстояние между двумя картинками равно : 0.7611444592475891
Косинусное расстояние между двумя картинками равно : 0.8296974897384644
Косинусное расстояние между двумя картинками равно : 0.7758249640464783
Косинусное расстояние между двумя картинками равно : 0.753829836845398
Косинусное расстояние между двумя картинками равно : 0.761905312538147
Косинусное расстояние между двумя картинками равно : 0.6724218130111694
Косинусное расстояние между двумя картинками равно : 0.7024937272071838
Косинусное расстояние между двумя картинками равно : 0.7383933067321777
Косинусное расстояние между двумя картинками равно : 0.81084835529

In [ ]:
# две картинки одинаковые но одна увеличенная без resize
compare_two_pictures('/content/sample_data/9.png', '/content/sample_data/9.big.png', model)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9297694563865662
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
# две картинки одинаковые но одна увеличенная с resize
compare_two_pictures_other1('/content/sample_data/9.png', '/content/sample_data/9.big.png', model)

Косинусное расстояние между двумя картинками равно : 0.9998390078544617
Совпадение соответствующих feature map двух картинок = 0.9765625 %


In [ ]:
# две картинки одинаковые но одна чернобелая без resize
compare_two_pictures('/content/sample_data/9.png', '/content/sample_data/9black.png', model)

(1, 512)
torch.Size([1, 3, 438, 427])
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9763650894165039
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
# две картинки одинаковые но одна чернобелая с resize
compare_two_pictures_other1('/content/sample_data/9.png', '/content/sample_data/9black.png', model)

Косинусное расстояние между двумя картинками равно : 0.9512357115745544
Совпадение соответствующих feature map двух картинок = 0.390625 %


In [ ]:
# сравниваем две одинаковые картинки одна с resize другая без
compare_two_pictures_other('/content/sample_data/9.png', '/content/sample_data/9.png', model)

Косинусное расстояние между двумя картинками равно : 0.9161692261695862
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
# сравниваем сканы двух разных документов
compare_two_pictures('/content/sample_data/скан1.jpg', '/content/sample_data/скан2.jpg', model)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9254958033561707
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
compare_two_pictures('/content/sample_data/скан1.jpg', '/content/sample_data/VqCifTlKe-A.jpg', model)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.8215687274932861
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
# две одинаковые картинки только одна имеет другой контраст
compare_two_pictures('sample_data/9.png', 'sample_data/9gl.png', model)

(1, 512)


FileNotFoundError: ignored

In [ ]:
# две одинаковые картинки только одна имеет другой контраст и поворот
compare_two_pictures('sample_data/9.png', 'sample_data/9gl_pov.png', model)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.7844823002815247
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
# две одинаковые картинки только одна черно-белая
compare_two_pictures('sample_data/9.png', 'sample_data/9black.png', model)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9986156821250916
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
# две одинаковые картинки только одна черно-белая
compare_two_pictures('/content/sample_data/красная площадь2.jpg', '/content/sample_data/красная площадь2_black.jpg', model)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9594357013702393
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
compare_two_pictures('/content/sample_data/красная площадь1.jpg', '/content/sample_data/красная площадь2_black.jpg', model)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.8560234904289246
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
# две одинаковые картинки только одна перевернута
compare_two_pictures('sample_data/9.png', 'sample_data/m_9.png', model)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9530755877494812
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
# две одинаковые картинки только одна растянута
compare_two_pictures('sample_data/9.png', '/content/sample_data/9.big.png', model)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9297694563865662
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
# две одинаковые картинки только в одной из них изменен 1 канал
compare_two_pictures('sample_data/9.png', 'sample_data/9.png', model, change_one_pixel=True)

(1, 512)
(1, 512)
Косинусное расстояние между двумя картинками равно : 0.9999993443489075
Совпадение соответствующих feature map двух картинок = 0.78125 %


In [ ]:
# две одинаковые картинки но под разными ракурсами
compare_two_pictures('sample_data/VqCifTlKe-A.jpg', 'sample_data/uKF9WyvZlbE.jpg', model)

Косинусное расстояние между двумя картинками равно : 0.9211031794548035
Совпадение соответствующих feature map двух картинок = 0.1953125 %


In [ ]:
# две фотографии с красной площадью в разное время суток и с разным ракурсом
compare_two_pictures('sample_data/красная площадь1.jpg', 'sample_data/красная площадь2.jpg', model)

Косинусное расстояние между двумя картинками равно : 0.7922313213348389
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
# сравнение картинок с собакой и кошкой
compare_two_pictures('sample_data/cat.jpg', 'sample_data/dog.jpg', model)

Косинусное расстояние между двумя картинками равно : 0.5034276247024536
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
# сравнение картинок с красной площадью и кошкой
compare_two_pictures('sample_data/cat.jpg', 'sample_data/красная площадь1.jpg', model)

Косинусное расстояние между двумя картинками равно : 0.48150527477264404
Совпадение соответствующих feature map двух картинок = 0.0 %


In [ ]:
compare_two_pictures('sample_data/красная площадь1.jpg', 'sample_data/красная площадь1.jpg', model)

(512,)
(512,)
Косинусное расстояние между двумя картинками равно : 0.9999998807907104
Совпадение соответствующих feature map двух картинок = 100.0 %


In [ ]:
!git clone https://github.com/laxmimerit/dog-cat-full-dataset.git

Cloning into 'dog-cat-full-dataset'...
remote: Enumerating objects: 25027, done.
remote: Total 25027 (delta 0), reused 0 (delta 0), pack-reused 25027
Receiving objects: 100% (25027/25027), 541.62 MiB | 24.84 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Updating files: 100% (25001/25001), done.


In [ ]:
path_dataset = Path("dog-cat-full-dataset/data/train")

In [ ]:
class BinaryClassification(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.model = torch.nn.Sequential(torch.nn.Linear(input_size, 64),
                               torch.nn.BatchNorm1d(64),
                               torch.nn.Dropout1d(0.2),
                               torch.nn.ReLU(),
                               torch.nn.Linear(64, 16),
                               torch.nn.BatchNorm1d(16),
                               torch.nn.Dropout1d(0.2),
                               torch.nn.ReLU(),
                               torch.nn.Linear(16, num_classes),
                               #torch.nn.Sigmoid())
                               torch.nn.Softmax())

    def get_weights(self):
        return self.weight

    def forward(self,x):
        x = self.model(x)
        return x

    def eval(self):
        self.model.eval()

In [ ]:
class CatDogDataset(Dataset):
    def __init__(self, path: Path, coder: ImageEncoder):
        self.path_dataset = path
        self.coder = coder
        self.path_dirs_images = [path_dir for path_dir in path_dataset.iterdir() if path_dir.is_dir()]
        self.path_images = [path_image for path_dir_images in self.path_dirs_images for path_image in path_dir_images.iterdir()]
        self.path_images = self.path_images[:500] + self.path_images[-500:]

    def __len__(self):
        return len(self.path_images)

    def __getitem__(self, index):
        image = Image.open(self.path_images[index])
        preprocessed = preprocess(image)
        output = self.coder(preprocessed)
        output = output.detach().cpu().numpy()
        image_feature_map512 = output[0]

        target = torch.zeros(2)

        if "dog" in self.path_images[index].stem:
            target[0] = 1
        else:
            target[1] = 1

        return image_feature_map512, target

In [ ]:
dataset = CatDogDataset(path_dataset, model)
trainloader = DataLoader(dataset, batch_size=20, shuffle=True)
model_classification = BinaryClassification(512, 2048, 2)
optimizer = torch.optim.Adam(model_classification.parameters(), lr=0.001)
loss = torch.nn.CrossEntropyLoss()

In [ ]:
def train_one_epoch(epoch, trainloader, optimizer, model, criterion, dataset):
    correct = 0
    epoch_loss = 0.0
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


    for i, data in enumerate(trainloader, 0):
        print(f"Батч № {i+1}")
        x, y = data
        # print(y)
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        model = model.to(device)
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        epoch_loss += loss.item()
        _, predicted = outputs.max(1)

        for j, i in enumerate(y.detach().cpu().numpy()):
          if list(i) == list([1.0, 0.0]):
              y_ = 0
          else:
              y_ = 1
          if y_ == int(predicted[j]):
              correct += 1

        optimizer.step()

    print(f'TRAIN [{epoch + 1}] loss: {epoch_loss / 50:.3f} accuracy: {correct / len(trainloader.dataset):.3f}')

In [ ]:
for epoch in range(50):
    train_one_epoch(epoch, trainloader, optimizer, model_classification, loss, dataset)

Батч № 1


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Батч № 2
Батч № 3
Батч № 4
Батч № 5
Батч № 6
Батч № 7
Батч № 8
Батч № 9
Батч № 10
Батч № 11
Батч № 12
Батч № 13
Батч № 14
Батч № 15
Батч № 16
Батч № 17
Батч № 18
Батч № 19
Батч № 20
Батч № 21
Батч № 22
Батч № 23
Батч № 24
Батч № 25
Батч № 26
Батч № 27
Батч № 28
Батч № 29
Батч № 30
Батч № 31
Батч № 32
Батч № 33
Батч № 34
Батч № 35
Батч № 36
Батч № 37
Батч № 38
Батч № 39
Батч № 40
Батч № 41
Батч № 42
Батч № 43
Батч № 44
Батч № 45
Батч № 46
Батч № 47
Батч № 48
Батч № 49
Батч № 50
TRAIN [1] loss: 0.549 accuracy: 0.773
Батч № 1
Батч № 2
Батч № 3
Батч № 4
Батч № 5
Батч № 6
Батч № 7
Батч № 8
Батч № 9
Батч № 10
Батч № 11
Батч № 12
Батч № 13
Батч № 14
Батч № 15
Батч № 16
Батч № 17
Батч № 18
Батч № 19
Батч № 20
Батч № 21
Батч № 22
Батч № 23
Батч № 24
Батч № 25
Батч № 26
Батч № 27
Батч № 28
Батч № 29
Батч № 30
Батч № 31
Батч № 32
Батч № 33
Батч № 34
Батч № 35
Батч № 36
Батч № 37
Батч № 38
Батч № 39
Батч № 40
Батч № 41
Батч № 42
Батч № 43
Батч № 44
Батч № 45
Батч № 46
Батч № 47
Батч № 48
Батч № 49

KeyboardInterrupt: ignored

In [ ]:
image = Image.open("dog-cat-full-dataset/data/test/dogs/dog.10025.jpg")
preprocessed = preprocess(image)
output = model(preprocessed)
output = output.detach().cpu().numpy()

In [ ]:
model_classification.eval()
model_classification(torch.Tensor(output).cuda())

tensor([[0.9407, 0.0593]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [ ]:
# на выходе одно значение
class BinaryClassification(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.model = torch.nn.Sequential(torch.nn.Linear(input_size, 64),
                               torch.nn.BatchNorm1d(64),
                               torch.nn.Dropout1d(0.2),
                               torch.nn.ReLU(),
                               torch.nn.Linear(64, 16),
                               torch.nn.BatchNorm1d(16),
                               torch.nn.Dropout1d(0.2),
                               torch.nn.ReLU(),
                               torch.nn.Linear(16, num_classes),
                               torch.nn.Sigmoid())

    def get_weights(self):
        return self.weight

    def forward(self, x):
        x = self.model(x)
        return x

    def eval(self):
        self.model.eval()

In [ ]:
class CatDogDataset(Dataset):
    def __init__(self, path: Path, coder: ImageEncoder):
        self.path_dataset = path
        self.coder = coder
        self.path_dirs_images = [path_dir for path_dir in path_dataset.iterdir() if path_dir.is_dir()]
        self.path_images = [path_image for path_dir_images in self.path_dirs_images for path_image in path_dir_images.iterdir()]
        self.path_images = self.path_images[:500] + self.path_images[-500:]

    def __len__(self):
        return len(self.path_images)

    def __getitem__(self, index):
        image = Image.open(self.path_images[index])
        preprocessed = preprocess(image)
        output = self.coder(preprocessed)
        output = output.detach().cpu().numpy()
        image_feature_map512 = output[0]

        if "dog" in self.path_images[index].stem:
            target = 0
        else:
            target = 1

        return image_feature_map512, np.float32(target)

In [ ]:
dataset = CatDogDataset(path_dataset, model)
trainloader = DataLoader(dataset, batch_size=20, shuffle=True)
model_classification = BinaryClassification(512, 2048, 1)
optimizer = torch.optim.Adam(model_classification.parameters(), lr=0.001)
loss = torch.nn.BCELoss()

In [ ]:
def train_one_epoch(epoch, trainloader, optimizer, model, criterion, dataset):
    correct = 0
    epoch_loss = 0.0
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


    for i, data in enumerate(trainloader, 0):
        print(f"Батч № {i+1}")
        x, y = data
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        model = model.to(device)
        outputs = model(x)
        loss = criterion(outputs, y.unsqueeze(-1))
        loss.backward()
        epoch_loss += loss.item()
        predicted = outputs

        for target, pred in zip(y.detach().cpu().numpy(), predicted.detach().cpu().numpy()):
            if float(pred) > 0.7:
                pred = 1
            else:
                pred = 0

            if pred == int(target):
                correct += 1

        optimizer.step()

    print(f'TRAIN [{epoch + 1}] loss: {epoch_loss / 50:.3f} accuracy: {correct / len(trainloader.dataset):.3f}')

In [ ]:
for epoch in range(50):
    train_one_epoch(epoch, trainloader, optimizer, model_classification, loss, dataset)

Батч № 1
Батч № 2
Батч № 3
Батч № 4
Батч № 5
Батч № 6
Батч № 7
Батч № 8
Батч № 9
Батч № 10
Батч № 11
Батч № 12
Батч № 13
Батч № 14
Батч № 15
Батч № 16
Батч № 17
Батч № 18
Батч № 19
Батч № 20
Батч № 21
Батч № 22
Батч № 23
Батч № 24
Батч № 25
Батч № 26
Батч № 27
Батч № 28
Батч № 29
Батч № 30
Батч № 31
Батч № 32
Батч № 33
Батч № 34
Батч № 35
Батч № 36
Батч № 37
Батч № 38
Батч № 39
Батч № 40
Батч № 41
Батч № 42
Батч № 43
Батч № 44
Батч № 45
Батч № 46
Батч № 47
Батч № 48
Батч № 49
Батч № 50
TRAIN [1] loss: 0.419 accuracy: 0.736
Батч № 1
Батч № 2
Батч № 3
Батч № 4
Батч № 5
Батч № 6
Батч № 7
Батч № 8
Батч № 9
Батч № 10
Батч № 11
Батч № 12
Батч № 13
Батч № 14
Батч № 15
Батч № 16
Батч № 17
Батч № 18
Батч № 19
Батч № 20
Батч № 21
Батч № 22
Батч № 23
Батч № 24
Батч № 25
Батч № 26
Батч № 27
Батч № 28
Батч № 29
Батч № 30
Батч № 31
Батч № 32
Батч № 33
Батч № 34
Батч № 35
Батч № 36
Батч № 37
Батч № 38
Батч № 39
Батч № 40
Батч № 41
Батч № 42
Батч № 43
Батч № 44
Батч № 45
Батч № 46
Батч № 47
Батч № 48


KeyboardInterrupt: ignored